# Implementing Proximal Policy Optimization

In [2]:
!git clone https://github.com/benelot/pybullet-gym lib/pybullet-gym
!pip install -e lib/pybullet-gym

Cloning into 'lib/pybullet-gym'...
remote: Enumerating objects: 804, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 804 (delta 21), reused 44 (delta 17), pack-reused 750
Receiving objects: 100% (804/804), 19.31 MiB | 20.08 MiB/s, done.
Resolving deltas: 100% (437/437), done.
Obtaining file:///data/home/vitya/DataPrep/experiments/od/PPO/lib/pybullet-gym
     |████████████████████████████████| 91.7 MB 1.1 MB/s eta 0:00:01
  Running setup.py develop for pybulletgym


In [1]:
import gym
import pybulletgym
import numpy as np

import torch
import torch.nn as nn
from torch.nn import init

In [18]:
from mujoco_wrappers import Normalize
from logger import TensorboardSummaries as Summaries

env = gym.make("HalfCheetahMuJoCoEnv-v0")
env = Normalize(Summaries(env))
env.unwrapped.seed(0)

print("observation space: ", env.observation_space, "\nobservations:", env.reset())
print(
    "action space: ", env.action_space, "\naction_sample: ", env.action_space.sample()
)

WalkerBase::__init__
argv[0]=
observation space:  Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf], (17,), float32) 
observations: [ 0.00092668  0.00026526 -0.00041955 -0.00079245  0.00034591 -0.00021478
  0.00033946 -0.00087684  0.          0.          0.          0.
  0.          0.          0.          0.          0.        ]
action space:  Box([-1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1.], (6,), float32) 
action_sample:  [ 0.4164298  -0.9497912  -0.6045943   0.6782092   0.27260754 -0.99954736]
argv[0]=


/data/home/vitya/DataPrep/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [19]:
def weights_init_orthogonal(m):
    classname = m.__class__.__name__
    # print(classname)
    if classname.find("Conv") != -1:
        init.orthogonal_(m.weight.data, gain=np.sqrt(2))
        init.constant_(m.bias.data, 0.0)
    elif classname.find("Linear") != -1:
        init.orthogonal_(m.weight.data, gain=np.sqrt(2))
        init.constant_(m.bias.data, 0.0)


class PPOModel(nn.Module):
    def __init__(self, obs_shape, n_actions, hidden_dim):
        super().__init__()
        self.n_actions = n_actions
        self.obs_shape = obs_shape
        self.hidden_dim = hidden_dim
        # conv2d_size_out(conv2d_size_out(conv2d_size_out(64, 3, 2), 3, 2), 3, 2)
        # Define your network body here. Please make sure agent is fully contained here
        self.backbone = nn.Sequential(
            nn.Linear(self.obs_shape, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.Tanh(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.Tanh(),
        )

        self.mean_head = nn.Linear(self.hidden_dim, self.n_actions)
        self.covariance_head = nn.Sequential(
            nn.Linear(self.hidden_dim, self.n_actions), nn.Softplus()
        )
        self.value_head = nn.Linear(self.hidden_dim, 1)

        # self.backbone.apply(weights_init_orthogonal)
        # self.mean_head.apply(weights_init_orthogonal)
        # self.covariance_head.apply(weights_init_orthogonal)
        # self.value_head.apply(weights_init_orthogonal)

    def forward(self, states):
        """
        input:
            states - tensor, (batch_size x features)
        output:
            mean - tensor, (batch_size x actions_dim)
            cov - tensor, (batch_size x actions_dim)
            V - tensor, critic estimation, (batch_size)
        """

        features = self.backbone(states)
        mean = self.mean_head(features)
        cov = self.covariance_head(features)
        V = self.value_head(features)

        # print(V.size())
        return mean, cov, V.squeeze()

This model will be wrapped by a `Policy`. The policy can work in two modes, but in either case 
it is going to return dictionary with string-type keys. The first mode is when the policy is 
used to sample actions for a trajectory which will later be used for training. In this case 
the flag `training` passed to `act` method is `False` and the method should return 
a `dict` with the following keys: 

* `"actions"`: actions to pass to the environment
* `"log_probs"`: log-probabilities of sampled actions
* `"values"`: value function $V^\pi(s)$ predictions.

We don't need to use the values under these keys for training, so all of them should be of type `np.ndarray`. This regime will be used to collect data.

When `training` is `True`, the model is training on a given batch of observations. In this
case it should return a `dict` with the following keys

* `"distribution"`: an instance of multivariate normal distribution (`torch.distributions.MultivariateNormal`)
* `"values"`: value function $V^\pi(s)$ prediction, tensor

The distinction about the modes comes into play depending on where the policy is used: if it is called from `EnvRunner`, 
the `training` flag is `False`, if it is called from `PPO`, the `training` flag is `True`. These classes 
will be described below.

In [20]:
from torch.distributions import MultivariateNormal

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class Policy:
    def __init__(self, model):
        self.model = model
        self.n_actions = self.model.n_actions

    def act(self, inputs, training=False):
        """
        input:
            inputs - numpy array if training is False, otherwise tensor, (batch_size x features)
            training - flag, bool
        output:
            if training is True, dict containing keys ['actions', 'log_probs', 'values']:
                `distribution` - MultivariateNormal, (batch_size x actions_dim)
                'values' - critic estimations, tensor, (batch_size)
            if training is False, dict containing keys ['actions', 'log_probs', 'values']:
                'actions' - selected actions, numpy, (batch_size)
                'log_probs' - log probs of selected actions, numpy, (batch_size)
                'values' - critic estimations, numpy, (batch_size)
        """
        # if training is false, input is numpy

        if not training:
            inputs = torch.FloatTensor(inputs).to(DEVICE)

        mean, cov, V = self.model(inputs)
        m = MultivariateNormal(mean, torch.diag_embed(cov))

        actions = m.sample()
        log_probs = m.log_prob(actions)

        if training:
            return {"distribution": m, "values": V}
        else:
            return {
                "actions": actions.detach().cpu().numpy(),
                "log_probs": log_probs.detach().cpu().numpy(),
                "values": V.detach().cpu().numpy(),
            }

We will use `EnvRunner` to perform interactions with an environment with a policy for a fixed number of timesteps. Calling `.get_next()` on a runner will return a trajectory &mdash; dictionary 
containing keys

* `"observations"`
* `"rewards"` 
* `"dones"`
* `"actions"`
* all other keys that you defined in `Policy` in `training=False` regime,

under each of these keys there is a `np.ndarray` of specified length $T$ &mdash; the size of partial trajectory. 

In [21]:
from runners import EnvRunner

Additionally, before returning a trajectory this runner can apply a list of transformations. 
Each transformation is simply a callable that should modify passed trajectory in-place.

In [22]:
import numpy as np


class AsArray:
    """
    Converts lists of interactions to ndarray.
    """

    def __call__(self, trajectory, last_observation):
        # Modifies trajectory inplace.
        # Just switches python lists to numpy arrays
        for k, v in trajectory.items():
            trajectory[k] = np.asarray(v)

Let's have a look at how this works.

In [23]:
model = PPOModel(17, 6, 16).to(DEVICE)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5, transforms=[AsArray()])

In [24]:
# generates new rollout
trajectory = runner.get_next()

In [25]:
# what is inside
print(trajectory.keys())

dict_keys(['actions', 'log_probs', 'values', 'observations', 'rewards', 'dones'])


In [26]:
trajectory["actions"]

array([[-0.9360738 ,  0.949878  , -0.6336447 , -0.09170941,  0.8897381 ,
         0.29848593],
       [ 0.3197962 ,  0.01018037, -0.5062505 , -0.0810163 , -0.28351927,
        -0.13856964],
       [ 0.6331702 , -0.39455462,  0.6109785 , -0.06013709, -0.63963383,
         0.49694097],
       [-0.40845338, -0.27975643, -0.20311242,  0.49331617, -0.11180159,
         1.0350164 ],
       [-1.0078186 , -0.76484746,  0.08241332, -0.3181476 , -0.97477615,
        -0.504859  ]], dtype=float32)

In [27]:
# Sanity checks
assert (
    "log_probs" in trajectory
), "Not found: policy didn't provide log_probs of selected actions"
assert "values" in trajectory, "Not found: policy didn't provide critic estimations"
assert trajectory["log_probs"].shape == (5,), "log_probs wrong shape"
assert trajectory["values"].shape == (5,), "values wrong shape"
assert trajectory["observations"].shape == (5, 17), "observations wrong shape"
assert trajectory["rewards"].shape == (5,), "rewards wrong shape"
assert trajectory["dones"].shape == (5,), "dones wrong shape"

In [28]:
# Here is what collected inside
{k: v.shape for k, v in trajectory.items()}

{'actions': (5, 6),
 'log_probs': (5,),
 'values': (5,),
 'observations': (5, 17),
 'rewards': (5,),
 'dones': (5,)}


The first is `GAE` that implements [Generalized Advantage Estimator](https://arxiv.org/abs/1506.02438).
In it you should add two keys to the trajectory: `"advantages"` and `"value_targets"`. In GAE the advantages
$A_t^{\mathrm{GAE}(\gamma,\lambda)}$ are essentially defined as the exponential 
moving average with parameter $\lambda$ of the regular advantages 
$\hat{A}^{(T)}(s_t) = \sum_{l=0}^{T-1-t} \gamma^l r_{t+l} + \gamma^{T} V^\pi(s_{T}) - V^\pi(s_t)$. 
The exact formula for the computation is the following

$$
A_{t}^{\mathrm{GAE}(\gamma,\lambda)} = \sum_{l=0}^{T-1-t} (\gamma\lambda)^l\delta_{t + l}^V, \, t \in [0, T)
$$
where $\delta_{t+l}^V = r_{t+l} + \gamma V^\pi(s_{t+l+1}) - V^\pi(s_{t+l})$. You can look at the 
derivation (formulas 11-16) in the paper. Don't forget to reset the summation on terminal
states as determined by the flags `trajectory["dones"]`. You can use `trajectory["values"]`
to get values of all observations except the most recent which is stored under 
 `trajectory["state"]["latest_observation"]`. For this observation you will need to call the policy 
 to get the value prediction.

Once you computed the advantages, you can get the targets for training the value function by adding 
back values:
$$
\hat{V}(s_{t+l}) = A_{t+l}^{\mathrm{GAE}(\gamma,\lambda)} + V(s_{t + l}),
$$
where $\hat{V}$ is a tensor of value targets that are used to train the value function. 

In [29]:
class GAE:
    """Generalized Advantage Estimator."""

    def __init__(self, policy, gamma=0.99, lambda_=0.95):
        self.policy = policy
        self.gamma = gamma
        self.lambda_ = lambda_

    def __call__(self, trajectory, last_observation):
        """
        This method should modify trajectory inplace by adding
        items with keys 'advantages' and 'value_targets' to it

        input:
            trajectory - dict from runner
            latest_observation - last state, numpy, (features)
        """
        num_steps, num_envs = np.vstack(trajectory["rewards"]).shape
        out_policy = self.policy.act(last_observation)
        values = out_policy["values"]

        advantages = np.zeros((num_steps, num_envs), dtype=np.float32)
        last_advantage = 0
        last_value = values

        for t in reversed(range(num_steps)):
            mask = 1.0 - trajectory["dones"][t]
            last_value = last_value * mask
            last_advantage = last_advantage * mask
            delta = (
                trajectory["rewards"][t]
                + self.gamma * last_value
                - trajectory["values"][t]
            )

            last_advantage = delta + self.gamma * self.lambda_ * last_advantage

            advantages[t] = last_advantage

            last_value = trajectory["values"][t]

        if num_envs == 1:
            advantages = advantages[:, 0]

        trajectory["advantages"] = advantages
        trajectory["value_targets"] = trajectory["values"] + trajectory["advantages"]

Let's run a small test just in case.

In [30]:
# tests
class DummyEnv:
    def __init__(self):
        self.unwrapped = None
        self.t = 0
        self.state = np.zeros(17)

    def reset(self):
        return self.state

    def step(self, a):
        r = [0, -100, 800][self.t]
        done = self.t == 2
        self.t = (self.t + 1) % 3
        return self.state, r, done, {}


class DummyPolicy:
    def act(self, s):
        return {"values": np.array(100), "actions": np.array([-0.42, 0.42])}


dummy_env = DummyEnv()
dummy_policy = DummyPolicy()
runner = EnvRunner(
    dummy_env,
    dummy_policy,
    nsteps=8,
    transforms=[AsArray(), GAE(dummy_policy, gamma=0.8, lambda_=0.5)],
)
trajectory = runner.get_next()

In [31]:
assert "advantages" in trajectory, "Not found: advantage estimation"
assert "value_targets" in trajectory, "Not found: targets for critic"
assert trajectory["advantages"].shape == (8,), "advantage wrong shape"
assert trajectory["value_targets"].shape == (8,), "value_targets wrong shape"
assert (
    trajectory["advantages"] == np.array([44, 160, 700, 44, 160, 700, -68, -120])
).all(), "advantage computation error"
assert (
    trajectory["value_targets"] == trajectory["advantages"] + 100
).all(), "value targets computation error"
print("Nice!")

Nice!


In [32]:
class Sampler:
    """Samples minibatches from trajectory for a number of epochs."""

    def __init__(self, runner, num_epochs, num_minibatches, transforms=None):
        self.runner = runner
        self.num_epochs = num_epochs
        self.num_minibatches = num_minibatches
        self.transforms = transforms or []

    def get_next(self):
        """
        Yields next minibatch (dict) for training with at least following keys:
                'observations' - numpy, (batch_size x features)
                'actions' - numpy, (batch_size x actions_dim)
                'advantages' - numpy, (batch_size)
                'log_probs' - numpy, (batch_size)
        """
        trajectory = self.runner.get_next()
        num_steps = len(trajectory["values"])

        splited = {}
        minibatch = {}

        for epoch in range(self.num_epochs):
            # shuffle dataset and separate it into minibatches
            # you can use any standard utils to do that

            idx = np.random.permutation(num_steps)
            for key in trajectory.keys():
                splited[key] = np.array_split(
                    trajectory[key][idx], self.num_minibatches
                )

            for i in range(self.num_minibatches):
                for key in splited.keys():
                    minibatch[key] = splited[key][i]

                # applying additional transforms
                for transform in self.transforms:
                    transform(minibatch)

                yield minibatch

A common trick to use with GAE is to normalize advantages for every minibatch, the following transformation does that. 

In [33]:
class NormalizeAdvantages:
    """Normalizes advantages to have zero mean and variance 1."""

    def __call__(self, batch):
        adv = batch["advantages"]
        adv = (adv - adv.mean()) / (adv.std() + 1e-8)
        batch["advantages"] = adv

In [34]:
class PyTorchify:
    """Moves everything to PyTorch"""

    def __call__(self, batch):
        for k, v in batch.items():
            batch[k] = torch.FloatTensor(v).to(DEVICE)

Finally, we can create our PPO runner! This is our pipeline of data collecting and generating mini-batches for our trainer!

In [35]:
def make_ppo_sampler(
    env,
    policy,
    num_runner_steps=2048,
    gamma=0.99,
    lambda_=0.95,
    num_epochs=10,
    num_minibatches=32,
):
    """Creates runner for PPO algorithm."""
    runner_transforms = [AsArray(), GAE(policy, gamma=gamma, lambda_=lambda_)]
    runner = EnvRunner(env, policy, num_runner_steps, transforms=runner_transforms)

    sampler_transforms = [NormalizeAdvantages(), PyTorchify()]
    sampler = Sampler(
        runner,
        num_epochs=num_epochs,
        num_minibatches=num_minibatches,
        transforms=sampler_transforms,
    )
    return sampler



$$
J_{\pi}(s, a) = \frac{\pi_\theta(a|s)}{\pi_\theta^{\text{old}}(a|s)} \cdot A^{\mathrm{GAE}(\gamma,\lambda)}(s, a)
$$

$$
J_{\pi}^{\text{clipped}}(s, a) = \mathrm{clip}\left(
\frac{\pi_\theta(a|s)}{\pi_{\theta^{\text{old}}}(a|s)},
1 - \text{cliprange}, 1 + \text{cliprange}\right)\cdot A^{\mathrm{GAE(\gamma, \lambda)}}(s)\\
$$

$$
L_{\text{policy}} = -\frac{1}{T}\sum_{l=0}^{T-1}\min\left(J_\pi(s_{t + l}, a_{t + l}), J_{\pi}^{\text{clipped}}(s_{t + l}, a_{t + l})\right).
$$

The value loss is also modified:

$$
L_{V}^{\text{clipped}} = \frac{1}{T}\sum_{l=0}^{T-1} \max(l^{simple}(s_{t + l}), l^{clipped}(s_{t + l})),
$$
where $l^{simple}$ is your standard critic loss
$$
l^{simple}(s_{t + l}) = [V_\theta(s_{t+l}) - \hat{V}(s_{t + l})]^2
$$

and $l^{clipped}$ is a clipped version that limits large changes of the value function:
$$
l^{clipped}(s_{t + l}) = [
V_{\theta^{\text{old}}}(s_{t+l}) +
\text{clip}\left(
V_\theta(s_{t+l}) - V_{\theta^\text{old}}(s_{t+l}),
-\text{cliprange}, \text{cliprange}
\right) - \hat{V}(s_{t + l})] ^ 2
$$

In [36]:
from torch.nn.utils import clip_grad_norm_


class PPO:
    def __init__(
        self,
        policy,
        optimizer,
        sampler,
        cliprange=0.2,
        value_loss_coef=0.25,
        max_grad_norm=0.5,
    ):
        self.policy = policy
        self.optimizer = optimizer
        self.sampler = sampler
        self.cliprange = cliprange
        self.value_loss_coef = value_loss_coef
        self.max_grad_norm = max_grad_norm
        self.iteration = 0

    def write(self, name, val):
        """For logging purposes"""
        self.sampler.runner.write(name, val, self.iteration)

    def policy_loss(self, batch, act):
        """
        Computes and returns policy loss on a given minibatch.
        input:
            batch - dict from sampler, containing:
                'advantages' - advantage estimation, tensor, (batch_size)
                'actions' - actions selected in real trajectory, (batch_size)
                'log_probs' - probabilities of actions from policy used to collect this trajectory, (batch_size)
            act - dict from your current policy, containing:
                'distribution' - MultivariateNormal, (batch_size x actions_dim)
        output:
            policy loss - torch scalar
        """
        current_probs = act["distribution"].log_prob(batch["actions"])
        r = torch.exp(current_probs - batch["log_probs"])

        J = r * batch["advantages"]
        J_cliped = (
            torch.clip(r, 1 - self.cliprange, 1 + self.cliprange) * batch["advantages"]
        )

        loss_policy = -torch.mean(torch.min(J, J_cliped))

        ratio = (abs(r - 1) > self.cliprange).to(torch.float).mean()
        entropy = act["distribution"].entropy()

        # additional logs: entropy, fraction of samples for which we zeroed gradient, max ratio
        self.write("additional/entropy", entropy.mean())
        self.write("additional/policy_loss_zeroed_gradient_fraction", ratio)
        self.write("additional/max_ratio", r.max())

        return loss_policy

    def value_loss(self, batch, act):
        """
        Computes and returns policy loss on a given minibatch.
        input:
            batch - dict from sampler, containing:
                'value_targets' - computed targets for critic, (batch_size)
                'values' - critic estimation from network that generated trajectory, (batch_size)
            act - dict from your current policy, containing:
                'values' - current critic estimation, tensor, (batch_size)
        output:
            critic loss - torch scalar
        """
        assert (
            batch["value_targets"].shape == act["values"].shape
        ), "Danger: your values and value targets have different shape. Watch your broadcasting!"

        critic_loss = (act["values"] - batch["value_targets"].detach()) ** 2
        critic_loss_cliped = (
            batch["values"]
            + torch.clip(
                (act["values"] - batch["values"]), -self.cliprange, self.cliprange
            )
            - batch["value_targets"].detach()
        ) ** 2

        loss = torch.mean(torch.maximum(critic_loss, critic_loss_cliped))

        ratio = (
            (abs((act["values"] - batch["values"])) > self.cliprange)
            .to(torch.float)
            .mean()
        )

        # additional logs: average value predictions, fraction of samples for which we zeroed gradient
        self.write("additional/value_predictions", act["values"].mean())
        self.write("additional/value_loss_zeroed_gradient_fraction", ratio)

        return loss

    def loss(self, batch):
        """Computes loss for current batch"""

        # let's run our current policy on this batch
        act = self.policy.act(batch["observations"], training=True)

        # compute losses
        # note that we don't need entropy regularization for this env.
        policy_loss = self.policy_loss(batch, act)
        critic_loss = self.value_loss(batch, act)

        # log all losses
        self.write("losses", {"policy loss": policy_loss, "critic loss": critic_loss})

        # Return scalar loss
        return policy_loss + self.value_loss_coef * critic_loss

    def step(self, batch):
        """Computes the loss function and performs a single gradient step for this batch."""

        loss = self.loss(batch)
        loss.backward()

        clip_grad_norm_(self.policy.model.parameters(), self.max_grad_norm)

        self.optimizer.step()
        self.optimizer.zero_grad()

        total_norm = 0
        for p in self.policy.model.parameters():
            param_norm = p.grad.detach().data.norm(2)
            total_norm += param_norm.item() ** 2
        total_norm = total_norm**0.5

        # do not forget to clip gradients using self.max_grad_norm
        # and log gradient norm
        self.write("gradient norm", total_norm)

        # this is for logging
        self.iteration += 1

In [37]:
model = PPOModel(17, 6, 64).to(DEVICE)
policy = Policy(model)
sampler = make_ppo_sampler(env, policy)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, eps=1e-5)
ppo = PPO(policy, optimizer, sampler)

In [ ]:
from tqdm import tqdm

epochs = 500

for i in range(epochs):
    for minibatch in sampler.get_next():
        ppo.step(minibatch)

In [127]:
# save your model just in case


def save(model, env, name):
    torch.save(model.state_dict(), name)
    np.save(name + "_mean_ob", env.obs_rmv.mean)
    np.save(name + "_var_ob", env.obs_rmv.var)
    np.save(name + "_count_ob", env.obs_rmv.count)


save(model, env, "PPO")

In [38]:
# loading the model


def load(model, env, name):
    model.load_state_dict(torch.load(name))
    env.obs_rmv.mean = np.load(name + "_mean_ob.npy")
    env.obs_rmv.var = np.load(name + "_var_ob.npy")
    env.obs_rmv.count = np.load(name + "_count_ob.npy")


load(model, env, "PPO")

## Evaluation

In [39]:
def evaluate(env, policy, n_games=1, t_max=1000):
    """
    Plays n_games and returns rewards and rendered games
    """
    rewards = []

    for _ in range(n_games):
        s = env.reset()

        R = 0
        for _ in range(t_max):
            action = policy.act(np.array([s]))["actions"][0]

            s, _, done, info = env.step(action)

            # remember that we used a wrapper that normalizes reward
            # original reward per step comes here
            R += info["original reward"]

            if done:
                break

        rewards.append(R)
    return np.array(rewards)

In [40]:
# evaluation will take some time!
sessions = evaluate(env, policy, n_games=20)
score = sessions.mean()
print(f"Your score: {score}")

assert score >= 1000, "Needs more training?"
print("Well done!")

Your score: 1508.9040631068149
Well done!


## Record

In [41]:
# let's hope this will work
# don't forget to pray
env = gym.wrappers.Monitor(env, directory="videos", force=True)

In [42]:
# record sessions
# note that t_max is 300, so collected reward will be smaller than 1000
evaluate(env, policy, n_games=1, t_max=300)

array([391.43007187])

In [43]:
env.close()